In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
def floats_to_bits(x):
    return np.unpackbits(np.frombuffer(x.tobytes(), dtype=np.uint8))

def bits_to_floats(x):
    return np.frombuffer(np.packbits(x).tobytes(), dtype=np.float)

We expect these results

In [ ]:
m = 0.1
g = 9.8
fval = m*g/4
f = fval*np.ones((4,), dtype=np.float)

In [ ]:
xi = tf.placeholder(tf.float32, [768, None])
W1 = tf.Variable(np.random.randn(768, 768), dtype=tf.float32)
W2 = tf.Variable(np.random.randn(256, 768), dtype=tf.float32)
eta = tf.placeholder(tf.float32, [256, None])

x = tf.matmul(W1, xi)
y = tf.nn.relu(x)
p = tf.matmul(W2, y)
z = tf.nn.sigmoid(p)
loss = tf.reduce_sum(tf.square(eta - z))

In [ ]:
init = tf.global_variables_initializer()

with tf.Session() as s:
    s.run(init)
    ll = s.run(loss, {xi: floats_to_bits(np.array([[1,1,1,0,0,0,1,1,1,0,0,0],
                                                   [-1,-1,-1,0,0,0,-1,-1,-1,0,0,0]])).reshape(768,2),
                      eta: floats_to_bits(np.array([f, f])).reshape(256,2)})

In [ ]:
ll

In [ ]:
with tf.Session() as s:
    s.run(init)
    gg = s.run(tf.gradients(loss, [W1, W2]),
               {xi: floats_to_bits(np.array([[1,1,1,0,0,0,1,1,1,0,0,0],
                                             [-1,-1,-1,0,0,0,-1,-1,-1,0,0,0]])).reshape(768,2),
                      eta: floats_to_bits(np.array([f, f])).reshape(256,2)})
    

In [ ]:
len(gg)

In [ ]:
xi_tr = np.random.randn(12, 1000)

In [ ]:
f_tr = fval*np.ones((4, 1000))

In [ ]:
with tf.Session() as s:
    s.run(init)
    ll = s.run(loss, {xi: floats_to_bits(xi_tr).reshape(768,1000),
                      eta: floats_to_bits(f_tr).reshape(256,1000)})

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(0.00001)
train = optimizer.minimize(loss)

In [ ]:
with tf.Session() as s:
    s.run(init)
    for i in range(10):
        print('Round', i+1)
        xi_tr = np.random.randn(12, 1000)
        f_tr = fval*np.ones((4, 1000))
        for j in range(1000):
            if j % 100 == 0:
                print(j, '/ 1000')
                print(s.run(loss, {xi: floats_to_bits(np.array([1,1,1,0,0,0,1,1,1,0,0,0])).reshape(768,1),
                                   eta: floats_to_bits(np.array([fval, fval, fval, fval])).reshape(256,1)}))
            s.run(train, {xi: floats_to_bits(xi_tr).reshape(768,1000),
                          eta: floats_to_bits(f_tr).reshape(256,1000)})
            
        # Check that the norm is getting lower
        zz = s.run(z, {xi: floats_to_bits(np.array([1,1,1,0,0,0,1,1,1,0,0,0])).reshape(768,1)})
        print(bits_to_floats(zz.reshape(256).astype(np.uint8)))
        
    # Check a predicted result
    zz = s.run(z, {xi: floats_to_bits(np.array([1,1,1,0,0,0,1,1,1,0,0,0])).reshape(768,1)})

In [ ]:
bits_to_floats(zz.reshape(256).astype(np.uint8))